In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score,accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import scale, LabelEncoder
from sklearn.model_selection import GridSearchCV

from lightgbm import LGBMClassifier
import lightgbm as lbg
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc

import re

dataset = pd.read_csv("application_train.csv")

num_dataset = pd.get_dummies(dataset)

In [2]:
data = num_dataset.drop('TARGET', 1)
target = num_dataset['TARGET']

X_train, X_test, y_train, y_test = train_test_split(data,target ,test_size = .2, random_state=42,shuffle=True)

train_data = lbg.Dataset(X_train,y_train)
test_data = lbg.Dataset(X_test,y_test)

PARAMS = {'max_depth':6,
          'num_leaves':21,
          'bagging_fraction':0.7,
          'feature_fraction':0.7,
          'boosting':'dart',
          'num_iterations':300,
          'early_stopping_rounds':30,
          'objective':'binary',
          'metric':'auc',
          'is_unbalance':True,
          'verbose':0
          }



In [ ]:
model = lbg.train(PARAMS,train_data,valid_sets=test_data)

pred = model.predict(X_test)

In [ ]:
### GRID SEARCH WIP ###
SEARCH_PARAMS = {'max_depth':[4,5,6,-1],
                 'num_leaves':[30,31,32],
                 'bagging_fraction':[0.7,1],
                 'feature_fraction':[0.7,1],
                 'boosting':['dart'],
                 'num_iterations':[100,200],
                 
                 'metric':['auc'],
                 
                 'verbose':[0]
                }

FIXED_PARAMS = {'objective':'binary',
                'metric':'auc',
                'is_unbalance':True,
                'verbose':0
                }
lb = lbg.LGBMClassifier()
clf = GridSearchCV(lb,SEARCH_PARAMS)
clf.fit(data,target)
GridSearchCV(estimator=LGBMClassifier(),
            param_grid=SEARCH_PARAMS)
sorted(clf.cv_results_.keys())

C:\Users\test\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [ ]:
print("done")

In [9]:
print(clf.cv_results_)

{'mean_fit_time': array([ 5.74104233,  6.80051551,  7.3956079 , 11.1917973 , 13.58786592,
       16.06812325,  6.57238922,  7.65988507,  7.89100275, 12.46709094,
       15.51386013, 18.16632404,  6.3175108 ,  8.05040617,  8.80361695,
       11.95489316, 16.08657866, 18.67307653,  6.65843787,  8.047789  ,
        9.24734173, 12.05134492, 16.04288688, 18.83310752]), 'std_fit_time': array([0.12054267, 0.16086356, 0.15176458, 0.40176915, 0.82722598,
       1.30594886, 0.27745221, 0.48987232, 0.48179719, 0.44803908,
       0.58056423, 0.66120944, 0.27217006, 0.41611918, 0.24568551,
       0.32869729, 0.6946925 , 0.75979931, 0.60735784, 0.29562015,
       0.4952443 , 0.4165231 , 0.5131266 , 0.63122819]), 'mean_score_time': array([0.36575875, 0.38500643, 0.38770466, 0.44377823, 0.45925417,
       0.51960459, 0.3844193 , 0.38890796, 0.39964662, 0.44868617,
       0.47742267, 0.49908295, 0.37351761, 0.40917401, 0.40177979,
       0.44429102, 0.51222939, 0.50545068, 0.3825841 , 0.39534178,
     

In [7]:
print(clf.best_estimator_)

LGBMClassifier(bagging_fraction=0.7, boosting='dart', feature_fraction=0.7,
               is_unbalance=True, max_depth=5, metric='auc', num_iterations=200,
               num_leaves=30, objective='binary', verbose=0)


In [10]:
print(clf.best_score_)

0.7479375888700632


In [11]:
kaggle_test = pd.read_csv("application_test.csv")
kaggle_test = pd.get_dummies(kaggle_test)

# Prevents an error on my machine
kaggle_test = kaggle_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# Adding columns to match training data
kaggle_test['CODE_GENDER_XNA'] = 0
kaggle_test['NAME_FAMILY_STATUS_Unknown'] = 0
kaggle_test['NAME_INCOME_TYPE_Maternityleave'] = 0

In [12]:
#estimating probabilities
final_pred = clf.best_estimator_.predict(kaggle_test)
final_pred = [max(x,0) for x in final_pred]

IDs = kaggle_test[['SK_ID_CURR']].astype('int64', copy=False)

submission_lgbm = pd.DataFrame({'SK_ID_CURR': IDs.SK_ID_CURR.astype('int64', copy=False).tolist(),
'TARGET': final_pred})

submission_lgbm.to_csv("lgbm_sub.csv", index=False)